# ARC Prize 2025 - Timing Management System

**Purpose:** Track competition execution time with 11:59:59 countdown

**Safety Margin:** Using 11h 59m 59s instead of full 12 hours

In [ ]:
import time
from datetime import datetime, timedelta
import json
import os

# Ensure working directory exists
os.makedirs('../working', exist_ok=True)
os.makedirs('../.claude/memory', exist_ok=True)

In [ ]:
class CompetitionTimer:
    """Manages timing for ARC Prize 2025 competition with 11:59:59 countdown"""
    
    def __init__(self, time_limit_seconds=43199):  # 11:59:59 = 43199 seconds
        """Initialize timer and record start time
        
        Args:
            time_limit_seconds: Total time allowed (default: 11h 59m 59s)
        """
        self.start_time = datetime.now()
        self.time_limit_seconds = time_limit_seconds
        self.start_timestamp = self.start_time.strftime("%Y-%m-%d %H:%M:%S")
        
        # Store in memory variable
        self.exam_start_datetime = self.start_time
        
        # Log to file (both locations)
        self._log_start_time()
        
        print(f"⏱️  Competition Timer Started")
        print(f"📅 Start Time: {self.start_timestamp}")
        print(f"⏳ Time Limit: 11:59:59 (safety margin)")
        print(f"🎯 End Time: {self.get_end_time()}")
    
    def _log_start_time(self):
        """Log start time to both working directory and Claude memory"""
        log_data = {
            "exam_start_datetime": self.start_timestamp,
            "time_limit_seconds": self.time_limit_seconds,
            "time_limit_formatted": "11:59:59",
            "end_datetime": self.get_end_time()
        }
        
        # Log to working directory (Kaggle standard)
        with open('../working/timer_log.json', 'w') as f:
            json.dump(log_data, f, indent=2)
        
        # Log to Claude memory
        with open('../.claude/memory/timer_log.json', 'w') as f:
            json.dump(log_data, f, indent=2)
    
    def get_elapsed_time(self):
        """Get elapsed time since start
        
        Returns:
            timedelta: Time elapsed
        """
        return datetime.now() - self.start_time
    
    def get_remaining_time(self):
        """Get remaining time until deadline
        
        Returns:
            timedelta: Time remaining (can be negative if exceeded)
        """
        elapsed = self.get_elapsed_time().total_seconds()
        remaining_seconds = self.time_limit_seconds - elapsed
        return timedelta(seconds=max(0, remaining_seconds))
    
    def get_end_time(self):
        """Get the deadline datetime
        
        Returns:
            str: Formatted end time
        """
        end_time = self.start_time + timedelta(seconds=self.time_limit_seconds)
        return end_time.strftime("%Y-%m-%d %H:%M:%S")
    
    def check_time_remaining(self, silent=True):
        """Check remaining time (silent by default)
        
        Args:
            silent: If False, print the status
            
        Returns:
            dict: Status information
        """
        now = datetime.now()
        elapsed = self.get_elapsed_time()
        remaining = self.get_remaining_time()
        
        status = {
            "current_time": now.strftime("%Y-%m-%d %H:%M:%S"),
            "elapsed": str(elapsed).split('.')[0],  # Remove microseconds
            "remaining": str(remaining).split('.')[0],
            "elapsed_seconds": elapsed.total_seconds(),
            "remaining_seconds": remaining.total_seconds(),
            "time_exceeded": remaining.total_seconds() == 0
        }
        
        if not silent:
            print(f"\n⏰ Time Check")
            print(f"   Now: {status['current_time']}")
            print(f"   Elapsed: {status['elapsed']}")
            print(f"   Remaining: {status['remaining']}")
            if status['time_exceeded']:
                print(f"   ⚠️  TIME LIMIT EXCEEDED!")
        
        return status
    
    def is_time_available(self, seconds_needed=60):
        """Check if there's enough time remaining for a task
        
        Args:
            seconds_needed: Minimum seconds needed
            
        Returns:
            bool: True if enough time remaining
        """
        remaining = self.get_remaining_time().total_seconds()
        return remaining >= seconds_needed
    
    def get_milestone_status(self, tasks_total=240, tasks_completed=0):
        """Get progress status for milestone reporting
        
        Args:
            tasks_total: Total number of tasks (default: 240 for ARC)
            tasks_completed: Number of tasks completed so far
            
        Returns:
            dict: Milestone status information
        """
        elapsed = self.get_elapsed_time().total_seconds()
        remaining = self.get_remaining_time().total_seconds()
        
        if tasks_completed > 0:
            avg_time_per_task = elapsed / tasks_completed
            estimated_total_time = avg_time_per_task * tasks_total
            on_track = estimated_total_time <= self.time_limit_seconds
        else:
            avg_time_per_task = 0
            estimated_total_time = 0
            on_track = True
        
        status = {
            "tasks_completed": tasks_completed,
            "tasks_remaining": tasks_total - tasks_completed,
            "progress_percentage": (tasks_completed / tasks_total * 100) if tasks_total > 0 else 0,
            "elapsed": str(timedelta(seconds=int(elapsed))),
            "remaining": str(timedelta(seconds=int(remaining))),
            "avg_seconds_per_task": round(avg_time_per_task, 2) if avg_time_per_task > 0 else 0,
            "on_track": on_track
        }
        
        return status
    
    def print_milestone(self, tasks_completed, tasks_total=240):
        """Print milestone status report
        
        Args:
            tasks_completed: Number of tasks completed
            tasks_total: Total tasks (default: 240)
        """
        status = self.get_milestone_status(tasks_total, tasks_completed)
        
        print(f"\n{'='*50}")
        print(f"📊 MILESTONE: {tasks_completed}/{tasks_total} Tasks Completed")
        print(f"{'='*50}")
        print(f"Progress: {status['progress_percentage']:.1f}%")
        print(f"Elapsed: {status['elapsed']}")
        print(f"Remaining: {status['remaining']}")
        print(f"Avg Time/Task: {status['avg_seconds_per_task']}s")
        print(f"Status: {'✅ On Track' if status['on_track'] else '⚠️  Behind Schedule'}")
        print(f"{'='*50}\n")

In [ ]:
# Initialize the competition timer (THIS IS THE FIRST CODE TO RUN)
timer = CompetitionTimer()

# Timer is now running silently in background
# Access via timer.check_time_remaining() anytime

In [ ]:
# Example: Check time silently (returns dict, no print)
status = timer.check_time_remaining(silent=True)
print(f"Current status: {status}")

In [ ]:
# Example: Print milestone after completing tasks
# Call this after every 10-20 tasks or at key checkpoints
timer.print_milestone(tasks_completed=0, tasks_total=240)

In [ ]:
# Example: Check if enough time for next task (default: 60s needed)
if timer.is_time_available(seconds_needed=180):  # Check for 3 minutes
    print("✅ Sufficient time for next task")
else:
    print("⚠️  Not enough time remaining")

## Timer Log Files

Start time is automatically logged to:
- `../working/timer_log.json` (Kaggle working directory)
- `../.claude/memory/timer_log.json` (Claude memory)

Both files contain:
- Exam start datetime
- Time limit (11:59:59)
- Calculated end time